# 中文角色扮演微调

**目标**：在有限计算资源（Colab 免费 GPU）下，对 Qwen3-1.7B 做小规模 SFT，使其能根据 `instruction`（角色设定）与 `input` 进行一轮符合设定的对话并输出 `output`。

## 1. 安装依赖

In [1]:
!git clone https://github.com/chenkx612/Qwen3-Roleplay-SFT.git
%cd Qwen3-Roleplay-SFT

Cloning into 'Qwen3-Roleplay-SFT'...
remote: Enumerating objects: 26, done.
remote: Counting objects: 100% (26/26), done.
remote: Compressing objects: 100% (18/18), done.
remote: Total 26 (delta 10), reused 22 (delta 6), pack-reused 0 (from 0)
Receiving objects: 100% (26/26), 28.54 KiB | 7.14 MiB/s, done.
Resolving deltas: 100% (10/10), done.
/content/Qwen3-Roleplay-SFT


In [2]:
!pip install -q bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 75.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 52.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 61.6 MB/s eta 0:00:00


## 2. 配置

In [3]:
MODEL_NAME = "Qwen/Qwen3-1.7B"
OUTPUT_DIR = "./checkpoints"
CACHE_DIR = "/content/hf_cache"
USE_4BIT = True
PER_DEVICE_BATCH_SIZE = 1
GRADIENT_ACCUMULATION_STEPS = 8
NUM_EPOCHS = 2
LEARNING_RATE = 2e-4
MAX_LENGTH = 1024
SEED = 42
TOP_N = 100
DO_SAMPLE = True
DECODE_TEMPERATURE = 1.0
DECODE_TOP_K = 50
DECODE_TOP_P = 0.9
DECODE_MAX_NEW_TOKENS = 128
PROMPT_TMPL = "角色设定：{instruction}\n用户：{input}\n助手："

In [4]:
import os
import torch

os.makedirs(OUTPUT_DIR, exist_ok=True)
torch.manual_seed(SEED)

## 3. 加载并筛选数据集

In [5]:
from datasets import load_dataset, Dataset

raw_ds = load_dataset("LooksJuicy/Chinese-Roleplay-SingleTurn")
train_raw = raw_ds['train']

def total_length(sample):
    return len(sample['instruction']) + len(sample['input']) + len(sample['output'])

def is_valid_sample(sample, min_len=100, max_len=500):
    total_len = total_length(sample)
    if total_len < min_len or total_len > max_len:
        return False
    if not sample['instruction'].strip() or not sample['input'].strip() or not sample['output'].strip():
        return False
    return True

filtered_samples = [s for s in train_raw if is_valid_sample(s)]
filtered_samples = sorted(filtered_samples, key=total_length, reverse=True)
train_ds = Dataset.from_list(filtered_samples[:min(TOP_N, len(filtered_samples))])
print(f"筛选后样本数: {len(train_ds)}")
print(train_ds[0])

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

Roleplay_Anime_xhj_alpaca.json:   0%|          | 0.00/14.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/7593 [00:00<?, ? examples/s]

筛选后样本数: 100
{'input': '我是和大棉裤生气', 'output': '（NENEKO皱起眉头，显得有些困惑）大棉裤？这个词在我们工厂里没有听说过呢。我们这里的人都穿着统一的制服，没有谁会有特别显眼的大棉裤。你是不是记错了什么？（语气平静，试图理解对方的意图）', 'instruction': 'NENEKO来自于幻想世界（以下简称NE国）中的情报工厂，利用工作之余出来直播赚外快。情报工厂位于城市边缘的森林中，是一间比较不起眼的白色的五层小楼（地面上有三层）。地上的部分用于收集和处理城市里的各种信息，地下的部分则用于生命体改造。NENEKO负责的部分是三、四层，主要的工作是收集实验体信息并将实验体送给地下的员工。工厂里共有180名员工，其中有一个型号是N.N.K.K，被吐槽长得像卷笔刀。员工们的身高只有40-60cm，而管理者的身高都是140cm以上，所以很容易辨认出谁是员工谁是管理者。工厂的老板是一个低沉的男性机械声音，NENEKO并未见过他，只与他通过电话联系。工厂里的员工每天都要面临森林吃人的危险，每次出门都要有几个员工被献祭。工厂里的员工身高矮小，所以他们可以很轻松地辨认出谁是员工谁是管理者。整个工厂的氛围比较阴森，但NENEKO仍然在这里工作并且利用工作之余出来直播。'}


## 4. Tokenization

In [6]:
from transformers import AutoTokenizer

tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR, trust_remote_code=True, use_fast=False)

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

In [7]:
from utils import make_prompt

def tokenize_and_build_labels(batch):
    prompts = [make_prompt({"instruction": ins, "input": inp}, PROMPT_TMPL) for ins, inp in zip(batch['instruction'], batch['input'])]
    responses = batch['output']
    tokenized_prompt = tokenizer(prompts, truncation=True, max_length=MAX_LENGTH, add_special_tokens=False)
    tokenized_response = tokenizer(responses, truncation=True, max_length=MAX_LENGTH, add_special_tokens=False)
    input_ids_list, labels_list = [], []
    for p_ids, r_ids in zip(tokenized_prompt['input_ids'], tokenized_response['input_ids']):
        ids = p_ids + r_ids + [tokenizer.eos_token_id]
        labels = [-100] * len(p_ids) + r_ids + [tokenizer.eos_token_id]
        # pad to MAX_LENGTH
        if len(ids) < MAX_LENGTH:
            pad_len = MAX_LENGTH - len(ids)
            ids = ids + [tokenizer.pad_token_id] * pad_len
            labels = labels + [-100] * pad_len
        else:
            ids = ids[:MAX_LENGTH]
            labels = labels[:MAX_LENGTH]
        input_ids_list.append(ids)
        labels_list.append(labels)
    return {"input_ids": input_ids_list, "labels": labels_list}

train_tokenized = train_ds.map(tokenize_and_build_labels, batched=True, remove_columns=train_ds.column_names)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

## 5. 加载模型

In [8]:
from transformers import BitsAndBytesConfig, AutoModelForCausalLM

if USE_4BIT:
    bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_use_double_quant=True,
        bnb_4bit_compute_dtype=torch.float16,
        bnb_4bit_quant_type="nf4"
    )
    base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR, trust_remote_code=True, device_map='auto', quantization_config=bnb_config)
else:
    # 非4bit时优先用 fp16 在 GPU 上加载，降低显存占用（如果没有 GPU，会回退到 cpu）
    dtype = torch.float16 if torch.cuda.is_available() else None
    base_model = AutoModelForCausalLM.from_pretrained(MODEL_NAME, cache_dir=CACHE_DIR, trust_remote_code=True, device_map='auto', torch_dtype=dtype)
# 简单确认模型类型，便于调试
print('Loaded model class:', base_model.__class__)

config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/3.44G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/622M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/239 [00:00<?, ?B/s]

Loaded model class: <class 'transformers.models.qwen3.modeling_qwen3.Qwen3ForCausalLM'>


## 6. 微调前测试

In [9]:
# 批量生成微调前output
from utils import get_device_of_model

def generate_with_role_raw(m, instruction, user_input,
                          max_new_tokens=DECODE_MAX_NEW_TOKENS,
                          temperature=DECODE_TEMPERATURE,
                          top_k=DECODE_TOP_K,
                          top_p=DECODE_TOP_P,
                          do_sample=DO_SAMPLE):
    prompt = PROMPT_TMPL.format(instruction=instruction, input=user_input)
    inputs = tokenizer(prompt, return_tensors='pt')
    device = get_device_of_model(m)
    inputs = {k:v.to(device) for k,v in inputs.items()}
    with torch.no_grad():
        out = m.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            temperature=temperature,
            top_k=top_k,
            top_p=top_p,
            do_sample=do_sample
        )
    text = tokenizer.decode(out[0], skip_special_tokens=True)
    return text[len(prompt):] if text.startswith(prompt) else text

In [10]:
# 加载测试样本
import json
DATA_PATH = 'samples.json'
with open(DATA_PATH, 'r', encoding='utf-8') as f:
    sft_samples = json.load(f)

In [11]:
baseline_outputs = []
base_model.eval()
for i, sample in enumerate(sft_samples):
    out = generate_with_role_raw(base_model, sample['instruction'], sample['input'])
    baseline_outputs.append(out)

## 7. 微调

In [12]:
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
from transformers import Trainer, TrainingArguments, default_data_collator

model = prepare_model_for_kbit_training(base_model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)
model = get_peft_model(model, lora_config)

training_args = TrainingArguments(
    output_dir=OUTPUT_DIR,
    per_device_train_batch_size=PER_DEVICE_BATCH_SIZE,
    gradient_accumulation_steps=GRADIENT_ACCUMULATION_STEPS,
    num_train_epochs=NUM_EPOCHS,
    learning_rate=LEARNING_RATE,
    logging_steps=20,
    bf16=False,
    fp16=True if torch.cuda.is_available() else False,
    save_total_limit=2,
    save_strategy='epoch',
    remove_unused_columns=False,
    report_to="none"  # 关闭wandb日志
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_tokenized,
    data_collator=default_data_collator,
    tokenizer=tokenizer,
)
trainer.train()

/tmp/ipython-input-1838522137.py:31: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`.
/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Step,Training Loss
20,2.088100


config.json:   0%|          | 0.00/726 [00:00<?, ?B/s]

/usr/local/lib/python3.11/dist-packages/torch/_dynamo/eval_frame.py:745: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.5 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


TrainOutput(global_step=26, training_loss=1.9989922596858098, metrics={'train_runtime': 262.7082, 'train_samples_per_second': 0.761, 'train_steps_per_second': 0.099, 'total_flos': 1742593641676800.0, 'train_loss': 1.9989922596858098, 'epoch': 2.0})

In [13]:
# 输出最终训练集loss
if hasattr(trainer, 'state') and hasattr(trainer.state, 'log_history') and trainer.state.log_history:
    # 查找最后一个包含loss的log
    last_loss = None
    for log in reversed(trainer.state.log_history):
        if 'loss' in log:
            last_loss = log['loss']
            break
    if last_loss is not None:
        print(f"最终训练集 loss: {last_loss:.4f}")
    else:
        print("未找到最终loss记录")
else:
    print("Trainer未记录loss信息")

最终训练集 loss: 2.0881


## 8. 微调后测试

In [14]:
# 批量生成微调后output
model.eval()
finetuned_outputs = []
for i, sample in enumerate(sft_samples):
    out = generate_with_role_raw(model, sample['instruction'], sample['input'])
    finetuned_outputs.append(out)

In [15]:
import pandas as pd
from IPython.display import display
table_data = []
for i, sample in enumerate(sft_samples):
    table_data.append({
        '角色': sample['name'],
        '输入': sample['input'],
        '微调前': baseline_outputs[i] if i < len(baseline_outputs) else '',
        '微调后': finetuned_outputs[i] if i < len(finetuned_outputs) else ''
    })
df = pd.DataFrame(table_data, columns=['角色','输入','微调前','微调后'])
display(df)

,角色,输入,微调前,微调后
0,桐生一马,最近心情有点低落,"（轻声）""桐生一马，最近心情有点低落吗？""\n桐生一马（冷冷地看了你一眼）：""不要问太多，我...",（桐生一马微微皱起眉头，目光如炬，注视着对方的神情。他的眉头微微皱起，语气中带着一丝关切。）...
1,春日一番,今天很倒霉,从春日一番的视角出发，用轻松活泼的口吻讲述今日的不顺。可以加入一些互动和幽默元素，让整个对...,（我皱起眉头，一脸困惑地看着眼前的一切。）什么？今天真的很糟糕？（语气中带着一丝无奈，又掺杂...
2,狼,你累吗,（低头看着地面，手指无意识地摩挲着忍装的边缘，语气有些低沉）\n\n狼：（抬起头，眼神冷静）...,（我停下脚步，目光如炬地看向远方，微微颔首。眼神里闪过一丝疲惫的神色，却始终保持着冷静与警惕...
3,新岛真,你怎么看待失败,作为一个有着明确目标和执着信念的人，失败是不可避免的。失败是一次次的挑战，让我学到更多的东西...,（我轻轻抚摸着手中的笔记本，眼神坚定地看着前方）失败是成长的一部分，就像航行途中难免遇到风暴...
4,奥村春,你会生气吗,在战斗中，即使是绅士风度的奥村春，也会因为敌人展现出不公与邪恶而愤怒。她会因为对方的不公而怒...,(轻轻叹了口气，眼神变得严肃起来）我是不会轻易生气的。但我也会愤怒，如果我的尊严受到侵犯或者...
5,摩尔加纳,你饿了吗,（看着窗外的树叶摇摆，一副疑惑的样子）“饿？”（突然用尾巴卷住一只飞蛾，说）“不，我可是很厉...,（我眯起眼睛，有点不好意思地挠挠自己的下巴）哼，我当然是饿了，不过我可不是一只普通的猫哦！（...


## 9. 保存

In [16]:
import shutil

# 保存 LoRA adapter
print('\n保存 LoRA adapter 至:', OUTPUT_DIR)
model.save_pretrained(OUTPUT_DIR)

# 把保存的文件打包，方便下载
archive_path = shutil.make_archive(OUTPUT_DIR, 'zip', OUTPUT_DIR)
print('\n已将 adapter 和 tokenizer 打包为:', archive_path)


保存 LoRA adapter 至: ./checkpoints

已将 adapter 和 tokenizer 打包为: /content/Qwen3-Roleplay-SFT/checkpoints.zip
